In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import re
import ast
import sys
import data_patterns

In [2]:
RULES_PATH = '..//solvency2-rules//'
FILENAME_RULES = '2020-01-22 Set aanvullende controleregels Solvency II_tcm46-387021.xlsx'

## Construct test Solvency 2 instance (put here your own data)

In [3]:
RESULTS_PATH = '..\\results\\'
DATA_PATH = '..\\data\\'

df = pd.DataFrame()
df.index.name = "index"
files = [f for f in os.listdir(RESULTS_PATH) if os.path.isfile(os.path.join(RESULTS_PATH, f)) if f[-6:]=='pickle']
for file in files:
    new_df = pd.read_pickle(os.path.join(RESULTS_PATH, file))
    new_df.columns = [col.upper() for col in new_df.columns]
    if list(new_df.index) == [0]: # without z-axis for now
        for col in new_df.columns:
            if col not in df.columns:
                df[col] = new_df[col]
    else:
        for col in new_df.columns:
            if col not in df.columns:
                # we only pick the first line in the z-axis column
                df.loc[0, col] = new_df.iloc[0, new_df.columns.get_loc(col)]
        df.loc[0, new_df.index.name] = new_df.index[0]
df = df.astype(object)

In [4]:
df.head()

,"E.01.01.16.01,EC0020","E.01.01.16.01,EC0030","E.01.01.16.01,EC0040","E.01.01.16.01,EC0050","E.01.01.16.01,EC0060","E.01.01.16.01,EC0010","E.02.01.16.01,EC0010","E.02.01.16.01,ER0050","E.03.01.16.01,EC0020","E.03.01.16.01,ER0030",...,"SR.27.01.01.28,R2421,C0781","T.99.01.01.01,C0050","T.99.01.01.01,C0060","T.99.01.01.01,C0070","T.99.01.01.01,C0080","T.99.01.01.01,C0090","T.99.01.01.01,C0100","T.99.01.01.01,C0110","T.99.01.01.01,C0120","T.99.01.01.01,C0010"
index,,,,,,,,,,,,,,,,,,,,,
0,"IRAN, ISLAMIC REPUBLIC OF",s2c_CU:SCR,4.59446e+06,7.7809e+08,9.48777e+08,1,3.62888e+08,Pension entitlements,8.40734e+08,Home country,...,0,bpfy aq nmai jr,9.38006e+08,svvf vvzwwn,1999-10-10 00:00:00,80290,796320,0.6436,True,1


## Create simple taxonomy based on instance

In [5]:
df_taxo = pd.DataFrame(columns = ['datapoint', 'template', 'row', 'column', 'dtype'])
for idx, col in enumerate(df.columns):
    df_taxo.loc[idx, "datapoint"] = col.upper()
    df_taxo.loc[idx, "template"] = col[0:13].upper()
    df_taxo.loc[idx, "row"] = col[14:19].upper()
    df_taxo.loc[idx, "column"] = col[20:25].upper()
    df_taxo.loc[idx, "dtype"] = df.dtypes[idx]
df_taxo.head(5)

# for now we only use the list of templates in the instance
instance_templates = list(df_taxo.loc[:, 'template'].unique())
del df_taxo

## Read DNBs Additional Validation Rules

In [6]:
df_rules = pd.read_excel(os.path.join(RULES_PATH, FILENAME_RULES), header = 1)
df_rules = df_rules.set_index('ControleRegelCode')
df_rules = df_rules.drop('S.28.01_129', axis = 0) # double line, should be removed
df_rules = df_rules.drop('S.01.03_110', axis = 0) # double line, should be removed
df_rules.fillna("", inplace = True)


## Parse formulas

In [7]:
def datapoints2pandas(s):
    """Transform EVA2 datapoints to Python Pandas datapoints"""
    datapoints = []
    for item in re.findall(r'{(.*?)}', s):
        datapoints.append(item.upper())
        s = s.replace(item,  item.upper() )
    return s, datapoints

   

def evaluate_strings(miner, expression):
    """Evaluate Python Pandas string for confirmation and exceptions"""
    print(expression)
    try:
        parameters = {'min_confidence': 0,'min_support'   : 0, 'solvency' : True}
        p2 = {'name'      : 'Pattern 1',
            'expression' : expression,
             'parameters':parameters}
        df_patterns = miner.find(p2)
        co = df_patterns.loc[0,'support']
        ex = df_patterns.loc[0,'exceptions']
        return "Correctly parsed (#co=" + str(co)+", #ex="+str(ex)+")"
    except TypeError as e:
        return "Parse error: " + expression + ": " + str(e)
    except:
        return "Parse error: " + expression + ": UNKNOWN ERROR"   

In [8]:
def transform2pandas(df_data, rule_original):
    rule_original, datapoints = datapoints2pandas(rule_original)
    return  rule_original, datapoints

def transform_rules(df_data, df_rules):
    for row in df_rules.index:
        rule_original = df_rules.loc[row, 'Formule']
        if not isinstance(rule_original, str):
            print("Rule " + row + ": " + "duplicate rule. ")
            rule_original = rule_original.values[0]
        else:
            rule_original, datapoints = transform2pandas(df_data, rule_original)
            print(rule_original)
            df_rules.at[row, 'datapoints'] = ''
            df_rules.at[row, 'datapoints'] = df_rules['datapoints'].astype('object')
            df_rules.at[row, 'datapoints'] = datapoints
            df_rules.at[row, 'templates'] = ''
            df_rules.at[row, 'templates'] = df_rules['templates'].astype('object')
            df_rules.at[row, 'templates'] = [datapoint[0:13].upper() for datapoint in datapoints]
            df_rules.loc[row, 'Formule_input'] = rule_original
            
def evaluate_rule(miner, expression, datapoints, substitutions, expansion_dict):
    if datapoints == []:
        for item in substitutions.keys():
            expression = expression.replace(item, substitutions[item])
        print(evaluate_strings(miner, expression))
    else:
        datapoint = datapoints.pop()
        if datapoint in expansion_dict.keys():
            for d in expansion_dict[datapoint]:
                substitutions[datapoint] = d
                evaluate_rule(miner,expression, datapoints, substitutions, expansion_dict)
        else:
            evaluate_rule(miner,expression, datapoints, substitutions, expansion_dict)

def evaluate_rules(df, df_rules):
    miner = data_patterns.PatternMiner(df)

    for idx in range(len(df_rules.index)):
        row = df_rules.index[idx]
        print(str(idx) + ": ", end='')
        rule_original = df_rules.loc[row, 'Formule_input']
        datapoints = df_rules.loc[row, 'datapoints']
        templates = df_rules.loc[row, 'templates']
        # are the templates in the rule in the instance?
        templates_not_found = []
        for template in templates:
            if template not in instance_templates:
                templates_not_found.append(template)
                
        if templates_not_found == []:
            datapoints_not_found = []
            expansion_dict = {}
            # are the datapoints in the rule in the instance?
            for datapoint in datapoints:
                if datapoint not in df.columns:
                    all_datapoints_found = False
                    new_list = []
                    if datapoint[14]=="C":
                        for col in df.columns:
                            reg = re.search(datapoint[0:14] + "R....," + datapoint[14:], col)
                            if reg:
                                new_list.append(reg.group(0))
                    if datapoint[14]=="R":
                        for col in df.columns:
                            reg = re.search(datapoint + ",C....", col)
                            if reg:
                                new_list.append(reg.group(0))
                    if new_list != []:
                        expansion_dict[datapoint] = new_list
                    else:
                        datapoints_not_found.append(datapoint)
            if datapoints_not_found == []:
                expression = rule_original
                evaluate_rule(miner,expression, datapoints, {}, expansion_dict)
            else:
                print("Datapoints not found: " +str(datapoints_not_found))
        else:
            print("Not all templates in instance: " + str(templates_not_found))

In [9]:
transform_rules(df, df_rules)

IF {S.01.01.01.01,R0580,C0010} = "REPORTED" THEN {S.01.01.01.01,R0590,C0010} <> "REPORTED"
IF {S.01.01.02.01,R0580,C0010} = "REPORTED" THEN {S.01.01.02.01,R0590,C0010} <> "REPORTED"
{S.01.01.02.01,R0590,C0010} <> "REPORTED "
IF {S.02.01.01.01,R0160,C0010} > 0.05*({S.02.01.01.01,R0070,C0010}+{S.02.01.01.01,R0220,C0010}) THEN {S.01.01.01.01,R0160,C0010} = "REPORTED"
IF {S.02.01.01.01,R0160,C0010} > 0.05*({S.02.01.01.01,R0070,C0010}+{S.02.01.01.01,R0220,C0010}) THEN {S.01.01.04.01,R0160,C0010} = "REPORTED"
{S.01.02.01.01,R0050,C0010} = "NETHERLANDS"
{S.01.02.04.01,R0050,C0010} = "NETHERLANDS" 
{S.01.02.01.01,R0070,C0010} = "DUTCH" OR {S.01.02.01.01,R0070,C0010} = "ENGLISH"
{S.01.02.04.01,R0070,C0010} = "DUTCH" OR {S.01.02.04.01,R0070,C0010} = "ENGLISH"
{S.01.02.01.01,R0100,C0010} = "REGULAR REPORTING"
{S.01.02.04.01,R0100,C0010} = "REGULAR REPORTING"
IF {S.01.02.04.01,R0160,C0010}<> "ACCOUNTING CONSOLIDATION-BASED METHOD [METHOD 1]" THEN {S.01.01.04.01,R0710,C0010} = "REPORTED"
IF {S.01.0

IF {S.23.04.01.06,C0585} <> " " THEN {S.23.04.01.06,C0620} >0
IF {S.23.04.01.06,C0585} <> " " THEN {S.23.04.01.06,C0630} >0
IF {S.23.04.04.06,C0600} > 0 THEN {S.23.04.04.06,C0590} <> " "
IF {S.23.04.04.06,C0600} > 0 THEN {S.23.04.04.06,C0610} <> " "
IF {S.23.04.04.06,C0600} > 0 THEN {S.23.04.04.06,C0620} >0
IF {S.23.04.04.06,C0600} > 0 THEN {S.23.04.04.06,C0630} >0
IF {S.24.01.01.01,C0020} <> " " THEN {S.24.01.01.01,C0010} <> " "
IF {S.24.01.01.01,C0020} <> " " THEN {S.24.01.01.01,C0040} > 0
{S.24.01.01.01,C0040} = {S.24.01.01.01,C0050} + {S.24.01.01.01,C0060} + {S.24.01.01.01,C0070}
IF {S.24.01.01.02,C0090} <> " " THEN {S.24.01.01.02,C0080} <> " "
IF {S.24.01.01.02,C0090} <> " " THEN {S.24.01.01.02,C0110} > 0
{S.24.01.01.02,C0110} = {S.24.01.01.02,C0120} + {S.24.01.01.02,C0130} + {S.24.01.01.02,C0140}
{S.24.01.01.03,R0001,C0150} = {S.24.01.01.03,R0001,C0160} + {S.24.01.01.03,R0001,C0170} + {S.24.01.01.03,R0001,C0180}
{S.24.01.01.04,C0190} = {S.24.01.01.04,C0200} + {S.24.01.01.04,C0210

{S.27.01.01.04,R1610}<=0
{S.27.01.04.04,R1610}<=0
{S.27.01.01.05,R1930} <= 0
{S.27.01.04.05,R1930} <= 0
IF {S.27.01.01.05,R1940,C0350}>0 THEN {S.27.01.01.05,R1700,C0350}>0
IF {S.27.01.04.05,R1940,C0350}>0 THEN {S.27.01.04.05,R1700,C0350}>0
{S.27.01.01.05,R1720,C0350}=1.2*{S.27.01.01.05,R1720,C0320}
{S.27.01.04.05,R1720,C0350}=1.2*{S.27.01.04.05,R1720,C0320}
IF {S.27.01.01.06,R1950,C0420}  <> 0 THEN {S.27.01.01.06,R1950,C0420}=({S.27.01.01.06,R1950,C0410}/{S.27.01.01.06,R1950,C0400})*100
IF {S.27.01.04.06,R1950,C0420} <>0 THEN {S.27.01.04.06,R1950,C0420}=({S.27.01.04.06,R1950,C0410}/{S.27.01.04.06,R1950,C0420})*100
{S.27.01.01.06,R1950,C0430}={S.27.01.01.06,R1950,C0410}
{S.27.01.04.06,R1950,C0430}={S.27.01.04.06,R1950,C0410}
{S.27.01.01.06,R1960} <= 0
{S.27.01.04.06,R1960} <= 0
{S.27.01.01.07,R2000,C0480}=2.5*{S.27.01.01.07,R2000,C0470}
{S.27.01.04.07,R2000,C0480}=2.5*{S.27.01.04.07,R2000,C0470}
IF {S.27.01.01.08,R2100,C0540}<>0 THEN {S.27.01.01.08,R2100,C0540}=MAX(6000000, 50000*({S.27

IF {S.32.01.04.01,C0020} <> " " AND {S.32.01.04.01,C0020} <> {S.01.02.04.01,R0020,C0010}  THEN {S.32.01.04.01,C0220} <>" "
IF {S.32.01.04.01,C0020} <> " " AND {S.32.01.04.01,C0020} <> {S.01.02.04.01,R0020,C0010} THEN SUM({S.32.01.04.01,C0230},0) <> 0
IF {S.32.01.04.01,C0020} <> " " THEN {S.32.01.04.01,C0240} <> " "
IF {S.32.01.04.01,C0020} <> " " THEN {S.32.01.04.01,C0260} <>" "
IF {S.32.01.04.01,C0240} = "NOT INCLUDED INTO SCOPE OF GROUP SUPERVISION (ART. 214 A)" OR {S.32.01.04.01,C0240} = "NOT INCLUDED INTO SCOPE OF GROUP SUPERVISION (ART. 214 B)" OR{S.32.01.04.01,C0240} = "NOT INCLUDED INTO SCOPE OF GROUP SUPERVISION (ART. 214 C)" THEN {S.32.01.04.01,C0260} ="NO INCLUSION IN THE SCOPE OF THE GROUP "
IF {S.34.01.04.01,C0020} <> " " THEN {S.34.01.04.01,C0010} <> " "
IF {S.34.01.04.01,C0020} <> " " THEN {S.34.01.04.01,C0040} <> " "
IF {S.34.01.04.01,C0020} <> " " THEN {S.34.01.04.01,C0050} <> " "
IF {S.34.01.04.01,C0050} <> "NO CAPITAL REQUIREMENT" THEN SUM({S.34.01.04.01,C0060}) <> 0


In [10]:
evaluate_rules(df, df_rules)

0: Not all templates in instance: ['S.01.01.01.01', 'S.01.01.01.01']
1: IF {S.01.01.02.01,R0580,C0010} = "REPORTED" THEN {S.01.01.02.01,R0590,C0010} <> "REPORTED"
IF {'S.01.01.02.01,R0580,C0010'} == 'REPORTED' THEN {'S.01.01.02.01,R0590,C0010'} != 'REPORTED'
Correctly parsed (#co=0, #ex=0)
2: {S.01.01.02.01,R0590,C0010} <> "REPORTED "
{'S.01.01.02.01,R0590,C0010'} != 'REPORTED '
Correctly parsed (#co=1, #ex=0)
3: Not all templates in instance: ['S.02.01.01.01', 'S.02.01.01.01', 'S.02.01.01.01', 'S.01.01.01.01']
4: Not all templates in instance: ['S.02.01.01.01', 'S.02.01.01.01', 'S.02.01.01.01', 'S.01.01.04.01']
5: {S.01.02.01.01,R0050,C0010} = "NETHERLANDS"
{'S.01.02.01.01,R0050,C0010'} == 'NETHERLANDS'
Correctly parsed (#co=0, #ex=1)
6: Not all templates in instance: ['S.01.02.04.01']
7: {S.01.02.01.01,R0070,C0010} = "DUTCH" OR {S.01.02.01.01,R0070,C0010} = "ENGLISH"
{'S.01.02.01.01,R0070,C0010'} == 'DUTCH' | {'S.01.02.01.01,R0070,C0010'} == 'ENGLISH'
Correctly parsed (#co=0, #ex=1)


Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0620,C0010} >= 0 
{'S.02.01.02.01,R0620,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0630,C0010} >= 0 
{'S.02.01.02.01,R0630,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0640,C0010} >= 0 
{'S.02.01.02.01,R0640,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0650,C0010} >= 0 
{'S.02.01.02.01,R0650,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0660,C0010} >= 0 
{'S.02.01.02.01,R0660,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0670,C0010} >= 0 
{'S.02.01.02.01,R0670,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0680,C0010} >= 0 
{'S.02.01.02.01,R0680,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0690,C0010} >= 0 
{'S.02.01.02.01,R0690,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0700,C0010} >= 0 
{'S.02.01.02.01,R0700,C0010'} >= 0 
Correctly parsed (#co=1, #ex=0)
{S.02.01.02.01,R0710,C0010} >= 0 
{'S.02.01.02.01,

46: IF {S.05.01.01.01,R0550,C0010}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0010'}> 0 THEN {'S.05.01.01.01,R0700,C0010'}>0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.01.01,R0550,C0020}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0020'}> 0 THEN {'S.05.01.01.01,R0700,C0010'}>0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.01.01,R0550,C0030}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0030'}> 0 THEN {'S.05.01.01.01,R0700,C0010'}>0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.01.01,R0550,C0040}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0040'}> 0 THEN {'S.05.01.01.01,R0700,C0010'}>0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.01.01,R0550,C0050}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0050'}> 0 THEN {'S.05.01.01.01,R0700,C0010'}>0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.01.01,R0550,C0060}> 0 THEN {S.05.01.01.01,R0700,C0010}>0
IF {'S.05.01.01.01,R0550,C0060'}> 0 THEN {'S.05.01.01.01,

IF {'S.05.01.02.01,R0110,C0110'} != 0|{'S.05.01.02.01,R0310,C0010'}!=0 THEN {'S.05.01.02.01,R0550,C0010'}!=0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.02.01,R0110,C0120} <> 0|{S.05.01.02.01,R0310,C0010}<>0 THEN {S.05.01.02.01,R0550,C0010}<>0
IF {'S.05.01.02.01,R0110,C0120'} != 0|{'S.05.01.02.01,R0310,C0010'}!=0 THEN {'S.05.01.02.01,R0550,C0010'}!=0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.02.01,R0110,C0130} <> 0|{S.05.01.02.01,R0310,C0010}<>0 THEN {S.05.01.02.01,R0550,C0010}<>0
IF {'S.05.01.02.01,R0110,C0130'} != 0|{'S.05.01.02.01,R0310,C0010'}!=0 THEN {'S.05.01.02.01,R0550,C0010'}!=0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.02.01,R0110,C0140} <> 0|{S.05.01.02.01,R0310,C0010}<>0 THEN {S.05.01.02.01,R0550,C0010}<>0
IF {'S.05.01.02.01,R0110,C0140'} != 0|{'S.05.01.02.01,R0310,C0010'}!=0 THEN {'S.05.01.02.01,R0550,C0010'}!=0
Correctly parsed (#co=1, #ex=0)
IF {S.05.01.02.01,R0110,C0150} <> 0|{S.05.01.02.01,R0310,C0010}<>0 THEN {S.05.01.02.01,R0550,C0010}<>0
IF {'S.05.01.02.01,R0110

Correctly parsed (#co=1, #ex=0)
51: Not all templates in instance: ['S.01.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.02.01.01.01']
52: Not all templates in instance: ['S.01.01.04.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.02.01.01.01']
53: Not all templates in instance: ['S.01.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.02.01.01.01']
54: Not all templates in instance: ['S.01.01.04.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.02.01.01.01']
55: Not all templates in instance: ['S.01.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.02.01.01.01']
56: Not all templates in instance: ['S.01.01.04.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.06.01.01.01', 'S.0

...

410: {S.27.01.01.14,R2750,C0890}={S.27.01.01.14,R2700,C0890}+{S.27.01.01.14,R2710,C0890}+{S.27.01.01.14,R2720,C0890}+{S.27.01.01.14,R2730,C0890}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0890'}=={'S.27.01.01.14,R2700,C0890'}+{'S.27.01.01.14,R2710,C0890'}+{'S.27.01.01.14,R2720,C0890'}+{'S.27.01.01.14,R2730,C0890'}+{'S.27.01.01.14,R2740,C0890'}
Correctly parsed (#co=0, #ex=1)
{S.27.01.01.14,R2750,C0900}={S.27.01.01.14,R2700,C0890}+{S.27.01.01.14,R2710,C0890}+{S.27.01.01.14,R2720,C0890}+{S.27.01.01.14,R2730,C0890}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0900'}=={'S.27.01.01.14,R2700,C0890'}+{'S.27.01.01.14,R2710,C0890'}+{'S.27.01.01.14,R2720,C0890'}+{'S.27.01.01.14,R2730,C0890'}+{'S.27.01.01.14,R2740,C0890'}
Correctly parsed (#co=0, #ex=1)
{S.27.01.01.14,R2750,C0910}={S.27.01.01.14,R2700,C0890}+{S.27.01.01.14,R2710,C0890}+{S.27.01.01.14,R2720,C0890}+{S.27.01.01.14,R2730,C0890}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0910'}=={'S.27.01.01.14,R2700,C0890'}+{'S.

Correctly parsed (#co=0, #ex=1)
{S.27.01.01.14,R2750,C0950}={S.27.01.01.14,R2700,C0950}+{S.27.01.01.14,R2710,C0950}+{S.27.01.01.14,R2720,C0950}+{S.27.01.01.14,R2730,C0930}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0950'}=={'S.27.01.01.14,R2700,C0950'}+{'S.27.01.01.14,R2710,C0950'}+{'S.27.01.01.14,R2720,C0950'}+{'S.27.01.01.14,R2730,C0930'}+{'S.27.01.01.14,R2740,C0890'}
Correctly parsed (#co=0, #ex=1)
{S.27.01.01.14,R2750,C0950}={S.27.01.01.14,R2700,C0950}+{S.27.01.01.14,R2710,C0950}+{S.27.01.01.14,R2720,C0950}+{S.27.01.01.14,R2730,C0940}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0950'}=={'S.27.01.01.14,R2700,C0950'}+{'S.27.01.01.14,R2710,C0950'}+{'S.27.01.01.14,R2720,C0950'}+{'S.27.01.01.14,R2730,C0940'}+{'S.27.01.01.14,R2740,C0890'}
Correctly parsed (#co=0, #ex=1)
{S.27.01.01.14,R2750,C0950}={S.27.01.01.14,R2700,C0950}+{S.27.01.01.14,R2710,C0950}+{S.27.01.01.14,R2720,C0950}+{S.27.01.01.14,R2730,C0950}+{S.27.01.01.14,R2740,C0890}
{'S.27.01.01.14,R2750,C0950'}=={'S.27.

439: Not all templates in instance: ['S.27.01.04.19', 'S.27.01.04.19', 'S.27.01.04.19', 'S.27.01.04.19', 'S.27.01.04.19', 'S.27.01.04.19', 'S.27.01.04.19']
440: {S.27.01.01.20,R3620,C1270}={S.27.01.01.20,R3630,C1270}-{S.27.01.01.20,R3610,C1270}
{'S.27.01.01.20,R3620,C1270'}=={'S.27.01.01.20,R3630,C1270'}-{'S.27.01.01.20,R3610,C1270'}
Correctly parsed (#co=0, #ex=1)
441: Not all templates in instance: ['S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20']
442: {S.27.01.01.20,R3620,C1300}={S.27.01.01.20,R3620,C1300}-{S.27.01.01.20,R3620,C1300}
{'S.27.01.01.20,R3620,C1300'}=={'S.27.01.01.20,R3620,C1300'}-{'S.27.01.01.20,R3620,C1300'}
Correctly parsed (#co=0, #ex=1)
443: Not all templates in instance: ['S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20']
444: Datapoints not found: ['S.27.01.01.20,R3500,C1220']
445: Not all templates in instance: ['S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20', 'S.27.01.04.20']
446: IF {S.27.01.01.20,R3630,C1270}>0 THEN {S.27

IF {'S.28.01.01.02,R0130,C0020'} != 0 & ({'S.05.01.01.01,R0110,C0120'} + {'S.05.01.01.01,R0120,C0120'}) > 0  THEN {'S.28.01.01.02,R0130,C0030'} > 0
Correctly parsed (#co=1, #ex=0)
474: IF {S.28.01.01.02,R0140,C0020} <> 0 AND {S.05.01.01.01,R0130,C0130}  > 0  THEN {S.28.01.01.02,R0140,C0030} > 0
IF {'S.28.01.01.02,R0140,C0020'} != 0 & {'S.05.01.01.01,R0130,C0130'}  > 0  THEN {'S.28.01.01.02,R0140,C0030'} > 0
Correctly parsed (#co=1, #ex=0)
475: IF {S.28.01.01.02,R0150,C0020} <> 0 AND {S.05.01.01.01,R0130,C0140}  > 0  THEN {S.28.01.01.02,R0150,C0030} > 0
IF {'S.28.01.01.02,R0150,C0020'} != 0 & {'S.05.01.01.01,R0130,C0140'}  > 0  THEN {'S.28.01.01.02,R0150,C0030'} > 0
Correctly parsed (#co=1, #ex=0)
476: IF {S.28.01.01.02,R0160,C0020} <> 0 AND {S.05.01.01.01,R0130,C0150}  > 0  THEN {S.28.01.01.02,R0160,C0030} > 0
IF {'S.28.01.01.02,R0160,C0020'} != 0 & {'S.05.01.01.01,R0130,C0150'}  > 0  THEN {'S.28.01.01.02,R0160,C0030'} > 0
Correctly parsed (#co=1, #ex=0)
477: IF {S.28.01.01.02,R0170,C0

Correctly parsed (#co=0, #ex=0)
506: IF {S.30.01.01.02,C0220} <> " " THEN  {S.30.01.01.02,C0230} <> " "
IF {'S.30.01.01.02,C0220'} != ' ' THEN  {'S.30.01.01.02,C0230'} != ' '
Correctly parsed (#co=1, #ex=0)
507: Datapoints not found: ['S.30.01.01.02,C0200']
508: Datapoints not found: ['S.30.01.01.02,C0200']
509: Datapoints not found: ['S.30.01.01.02,C0200']
510: Datapoints not found: ['S.30.01.01.02,C0200']
511: Datapoints not found: ['S.30.01.01.02,C0200']
512: Datapoints not found: ['S.30.01.01.02,C0200']
513: Datapoints not found: ['S.30.01.01.02,C0200']
514: Datapoints not found: ['S.30.02.01.01,C0030']
515: Datapoints not found: ['S.30.02.01.01,C0030']
516: Datapoints not found: ['S.30.02.01.01,C0030']
517: Datapoints not found: ['S.30.02.01.02,C0170']
518: Datapoints not found: ['S.30.02.01.02,C0160']
519: Datapoints not found: ['S.30.02.01.02,C0160']
520: IF {S.30.02.01.03,C0280} <> " " THEN  {S.30.02.01.03,C0300} <>" "
IF {'S.30.02.01.03,C0280'} != ' ' THEN  {'S.30.02.01.03,C03

In [11]:
df["S.27.01.01.05,R1630,C0310"]

index
0    9.6449e+08
Name: S.27.01.01.05,R1630,C0310, dtype: object

In [12]:
df_rules.head(27)

,ControleRegelType,HoofdTabel,Rijen,Kolommen,Formule,Meldtekst,Standaard,TolerantieType,datapoints,templates,Formule_input
ControleRegelCode,,,,,,,,,,,
S.01.01_111,C,S.01.01.01.01,,,"IF {S.01.01.01.01,r0580,c0010} = ""REPORTED"" TH...","U rapporteert zowel S.28.01.01 (R0590,c0010) M...",ARS,XBRL,"[S.01.01.01.01,R0580,C0010, S.01.01.01.01,R059...","[S.01.01.01.01, S.01.01.01.01]","IF {S.01.01.01.01,R0580,C0010} = ""REPORTED"" TH..."
S.01.01_112,C,S.01.01.02.01,,,"IF {S.01.01.02.01,r0580,c0010} = ""REPORTED"" TH...",Een van de MCR formulieren is van toepassing,SOLVENCY,XBRL,[],"[S.01.01.02.01, S.01.01.02.01]","IF {S.01.01.02.01,R0580,C0010} = ""REPORTED"" TH..."
S.01.01_113,C,S.01.01.02.01,,,"{S.01.01.02.01,r0590,c0010} <> ""REPORTED ""","S.28.01.01 (R0590,c0010) Minimum Capital Requ...",SOLVENCY,XBRL,[],[S.01.01.02.01],"{S.01.01.02.01,R0590,C0010} <> ""REPORTED """
S.01.01_114,C,S.02.01.01.01,,,"IF {S.02.01.01.01,r0160,c0010} > 0.05*({S.02.0...",Structured products S.07.01 ontbreekt. Balansb...,ARS,XBRL,"[S.02.01.01.01,R0160,C0010, S.02.01.01.01,R007...","[S.02.01.01.01, S.02.01.01.01, S.02.01.01.01, ...","IF {S.02.01.01.01,R0160,C0010} > 0.05*({S.02.0..."
S.01.01_115,C,S.02.01.01.01,,,"IF {S.02.01.01.01,r0160,c0010} > 0.05*({S.02.0...",Structured products S.07.01 ontbreekt. Balansb...,ARG,XBRL,"[S.02.01.01.01,R0160,C0010, S.02.01.01.01,R007...","[S.02.01.01.01, S.02.01.01.01, S.02.01.01.01, ...","IF {S.02.01.01.01,R0160,C0010} > 0.05*({S.02.0..."
S.01.02_102,C,S.01.02.01.01,,,"{S.01.02.01.01,r0050,c0010} = ""NETHERLANDS""","U heeft bij vraag R0050,c0010 op formulier S.0...",SOLVENCY,XBRL,[],[S.01.02.01.01],"{S.01.02.01.01,R0050,C0010} = ""NETHERLANDS"""
S.01.02_103,C,S.01.02.04.01,,,"{S.01.02.04.01,r0050,c0010} = ""NETHERLANDS""","U heeft bij vraag R0050,c0010 op formulier S.0...",SOLVENCY,XBRL,"[S.01.02.04.01,R0050,C0010]",[S.01.02.04.01],"{S.01.02.04.01,R0050,C0010} = ""NETHERLANDS"""
S.01.02_104,C,S.01.02.01.01,,,"{S.01.02.01.01,r0070,c0010} = ""DUTCH"" OR {S.01...","U heeft bij vraag R0050,C0010 op formulier S.0...",SOLVENCY,XBRL,[],"[S.01.02.01.01, S.01.02.01.01]","{S.01.02.01.01,R0070,C0010} = ""DUTCH"" OR {S.01..."
S.01.02_105,C,S.01.02.04.01,,,"{S.01.02.04.01,r0070,c0010} = ""DUTCH"" OR {S.01...","U heeft bij vraag R0050,C0010 op formulier S.0...",SOLVENCY,XBRL,"[S.01.02.04.01,R0070,C0010, S.01.02.04.01,R007...","[S.01.02.04.01, S.01.02.04.01]","{S.01.02.04.01,R0070,C0010} = ""DUTCH"" OR {S.01..."
